# **The Stress-Sighting Hypothesis**
## A Data-Driven Analysis of Global Events and Reports of the Unknown.

**The Stress-Sighting Hypothesis** has the project goal of investigating whether there is a meaningful correlation between the frequency of reported UFO sightings and periods of heightened cultural, political or global stress, using historical event data and publicly reported sightings. 


## User Story
Alex Holloway is an investigative journalist, known for in-depth features that combine cultural analysis with data storytelling. They work with both independent media outlets and major publishers, seeking to explore how society processes uncertainty — from political unrest to media myths.

Alex is planning to write an article on how Global Stress Events impact the number of UFO sightings, and has asked us to conduct our analysis, using the publicly available NUFORC (National UFO Reporting Centre) UFO Sightings dataset found [here](https://www.kaggle.com/datasets/NUFORC/ufo-sightings/data)

## Business Requirements
In an era shaped by information saturation, political polarisation, and global crises, public perception is increasingly complex and emotionally charged. For journalists, researchers, and communicators, understanding how people respond to uncertainty is as important as the events themselves.

This project explores the potential relationship between **reported UFO sightings** and **global stress events**, not to investigate extraterrestrial phenomena, but to examine whether these sightings reflect **underlying patterns of public anxiety, media influence, and cultural tension.**

The outcome is a data-driven dashboard designed to support those working at the intersection of **data**, **storytelling**, and **public insight**.

![Alex Holloway – Persona Card](../images/alex_holloway_persona_card.png)

### Alex's Requirements:

- **Reveal Patterns**

Alex needs to identify correlations between historical periods of stress and spikes in UFO reporting - fast, clearly and without technical issues. 

- **Narrative Context**

They want to explore not just *when* things happened, but *why it matters.* Explanatory text and annotations support deeper storytelling.

- **Usable Insights**

Our charts and summaries must be easy to extract for use in articles or reports, including explanatory captions and legends.

- **Trustworthy Structure**

The data pipeline must be transparent, ethical and well-documented to ensure and maintain credibility in their journalistic work.

### Value Proposition:
Our Dashboard must empower users like Alex to:
- Translate complex data into cultural insight
- Frame journalistic stories with empirical evidence
- Uncover social signals hiding in unconventional data
- Offer the audience a grounded perspective on how fear, media, and uncertainty intersect.

---

## Hypotheses

Our Hypotheses for this project are as follows:

### **Hypothesis 1:** 

**There is a positive correlation between the number of glabal stress events in a given year and the number of UFO sightings.**

### **Hypothesis 2:**

**Years with higher total stress severity scores are associated with a greater number of UFO sightings.**

### **Hypothesis 3:**

**Cultural media events, (such as the release of UFO-themed films or television series) correspond with noticeable short-term spikes in reported sightings.**

For the sake of brevity, we will not outline our validation approaches here, as this will be covered in a seperate notebook.

---

## Data Preparation and Cleaning

In this section we will look to extract our data and give consideration to how we will clean it in order to make it effective for analysis. 
Our first step is to load our datasets: